### import library

In [ ]:
!pip install torch torchvision timm albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

google drive connect

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### adjust dataset according to training process

In [ ]:
import os
import shutil
import random

# Define source and destination paths
source_dir = "/content/drive/MyDrive/intern/dataset"
destination_dir = "/content/dataset"

# Define train-validation split ratio
train_ratio = 0.8  # 80% training, 20% validation

# Create train and val directories
train_dir = os.path.join(destination_dir, "train")
val_dir = os.path.join(destination_dir, "val")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Iterate over class folders
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)

    if os.path.isdir(class_path):  # Ensure it's a directory
        images = os.listdir(class_path)
        random.shuffle(images)  # Shuffle for randomness

        # Split dataset
        split_idx = int(len(images) * train_ratio)
        train_images, val_images = images[:split_idx], images[split_idx:]

        # Create class directories in train and val
        train_class_dir = os.path.join(train_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)

        # Move images to respective folders
        for img in train_images:
            shutil.copy(os.path.join(source_dir, class_name, img), os.path.join(train_class_dir, img))

        for img in val_images:
            shutil.copy(os.path.join(source_dir, class_name, img), os.path.join(val_class_dir, img))

print("Dataset restructuring complete!")


Dataset restructuring complete!


import library for Vision Transformer model process

In [ ]:
import torch
import torch.nn as nn
import timm
import numpy as np
import torch
import torch.nn as nn
import timm
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from albumentations.pytorch import ToTensorV2
import cv2
import os
from PIL import Image
from PIL import Image

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.4' (you have '2.0.3'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
#Define transformations for training and validation
import albumentations as A
train_transforms=A.Compose([
    A.Resize(224,224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=(0.485,0.456,0.406),std=(0.229,0.224,0.225)),
    ToTensorV2(),
])

val_transforms=A.Compose([
    A.Resize(224,224),
    A.Normalize(mean=(0.485,0.456,0.406),std=(0.229,0.224,0.225)),
    ToTensorV2(),
])

# Custom dataset class for Albumentations
class AlbumentationsDataset(torch.utils.data.Dataset):
  def __init__(self,dataset,transform):
    self.dataset=dataset
    self.transform=transform

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self,idx):
    img,label=self.dataset[idx]
    img=np.array(img)
    img=self.transform(image=img)["image"]
    return img, label

# Define dataset paths
train_path="/content/dataset/train"
val_path="/content/dataset/val"

In [ ]:
import os
from torchvision import datasets


# Define dataset paths
train_path = "/content/dataset/train"
val_path = "/content/dataset/val"

# Load datasets directly, assuming the structure is correct
train_dataset = datasets.ImageFolder(root=train_path)
val_dataset = datasets.ImageFolder(root=val_path)



In [ ]:
#Apply transformations
train_dataset=AlbumentationsDataset(train_dataset,train_transforms)
val_dataset=AlbumentationsDataset(val_dataset,val_transforms)



In [ ]:
# Define DataLoaders
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True, num_workers=2)
val_loader=DataLoader(val_dataset,batch_size=32,shuffle=False,num_workers=2)




In [ ]:
print(f"Train samples : {len(train_dataset)},validation samples :{len(val_dataset)}")

Train samples : 1549,validation samples :395


In [ ]:
## Load & Fine-tuning vision transformers (ViT)
#Load pre-trained vision Transformer

model=timm.create_model("vit_base_patch16_224",pretrained=True,num_classes=15)

# Freeze backbone layers(except classifier)

for param in model.parameters():
  param.requires_grad=False # Freeze all layers
for param in model.head.parameters():
  param.requires_grad=True # Unfreeze classifier layer

# Move model to GPU if available
device =torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=model.to(device)

# Define loss and optimizer
criterion=nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.head.parameters(),lr=1e-4)

#Check if GPU is available
print(f"Using device:{device}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Using device:cuda


start train epochs

In [ ]:
import numpy as np

def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=100, patience=5):
    """
    Trains the model for a specified number of epochs with early stopping.

    Args:
        model: The model to train.
        train_loader: DataLoader for the training data.
        val_loader: DataLoader for the validation data.
        criterion: The loss function.
        optimizer: The optimizer to use.
        epochs: The number of epochs to train for (default: 100).
        patience: The number of epochs to wait before stopping if validation loss doesn't improve (default: 5).
    """
    best_val_loss = np.inf  # Initialize with a very large value
    epochs_without_improvement = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        print(f"Epoch {epoch + 1}/{epochs} - Train Loss: {train_loss:.4f} - Train Acc: {train_acc:.2f}%")

        # Validation step
        model.eval()
        val_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                correct += (outputs.argmax(1) == labels).sum().item()
                total += labels.size(0)

        val_acc = 100 * correct / total
        avg_val_loss = val_loss / len(val_loader)
        print(f"Validation Accuracy: {val_acc:.2f}% - Validation Loss: {avg_val_loss:.4f}\n")

        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f"Early stopping triggered after {epoch + 1} epochs.")
                break  # Exit the training loop

train_model(model, train_loader, val_loader, criterion, optimizer, epochs=100)

Epoch 1/100 - Train Loss: 15.8697 - Train Acc: 92.51%
Validation Accuracy: 88.35% - Validation Loss: 0.3735

Epoch 2/100 - Train Loss: 15.6239 - Train Acc: 92.19%
Validation Accuracy: 88.10% - Validation Loss: 0.3675

Epoch 3/100 - Train Loss: 14.8776 - Train Acc: 92.83%
Validation Accuracy: 88.61% - Validation Loss: 0.3603

Epoch 4/100 - Train Loss: 13.6789 - Train Acc: 93.67%
Validation Accuracy: 88.61% - Validation Loss: 0.3572

Epoch 5/100 - Train Loss: 13.2474 - Train Acc: 93.48%
Validation Accuracy: 88.86% - Validation Loss: 0.3519

Epoch 6/100 - Train Loss: 14.2240 - Train Acc: 92.90%
Validation Accuracy: 88.61% - Validation Loss: 0.3479

Epoch 7/100 - Train Loss: 13.5857 - Train Acc: 93.35%
Validation Accuracy: 89.37% - Validation Loss: 0.3428

Epoch 8/100 - Train Loss: 13.4034 - Train Acc: 92.90%
Validation Accuracy: 88.86% - Validation Loss: 0.3398

Epoch 9/100 - Train Loss: 12.8253 - Train Acc: 93.67%
Validation Accuracy: 89.11% - Validation Loss: 0.3366

Epoch 10/100 - Trai

save model in drive for future use

In [ ]:
# Save model to Google Drive (if mounted)
model_path = "/content/drive/MyDrive/vit_model.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved at {model_path}")


Model saved at /content/drive/MyDrive/vit_model.pth


load model and test sample data

In [ ]:
# Load trained model
model.load_state_dict(torch.load(model_path, map_location=device), strict=False)  # Add strict=False
model.eval()

# Function to predict class of an image
def predict_image(image_path, model, class_names):  # Add class_names argument
    # Ensure the image file exists
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")

    image = Image.open(image_path).convert("RGB")
    image = val_transforms(image=np.array(image))["image"].unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        class_id = torch.argmax(output, dim=1).item()

    # Get class name from class ID
    class_name = class_names[class_id]  # Get class name
    return class_name  # Return class name instead of ID

# Get class names from your dataset
class_names = train_dataset.dataset.classes

# Provide the correct path to your image
image_path = "/content/test_img.jpg"

# Test on the image, but ensure the image file exists first.
if os.path.exists(image_path):
  predicted_class_name = predict_image(image_path, model, class_names)  # Pass class_names
  print(f"Predicted class for {image_path}: {predicted_class_name}")
else:
  print(f"Error: Image file not found at {image_path}")

<ipython-input-24-1872ff1409dc>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device), strict=False)  # Add stri

Predicted class for /content/test_img.jpg: Tiger
